# Découpage d'un audio

Ce notebook vous permet de :

1. **Découper** un fichier audio de réunion en sous-fichier audio du même type de durée d'environ 1h. Il coupe l'audio en X sous-fichier de 1h après un blanc dans l'audio pour éviter une perte d'information.

Pour faire simple, votre fichier audio va être découper en sous-fichiers audio d'une heure environ si celui-ci dépasse une heure de longueur. Les sous-fichiers sont directement sauvegarder dans le dossier "IA pour maman"

## Pré-requis

- Un compte Google pour utiliser Google Colab.
- Avoir enregistré votre fichier audio (exemple : `Meeting.m4a`) dans votre Google Drive.

Suivez les instructions de chaque cellule pour monter votre Google Drive et indiquer le chemin de votre fichier audio.

**Pour lancer une cellule, cliquez une seule fois sur le bouton play en haut à gauche des cellules contenant du code.**


In [ ]:
# Monter Google Drive pour accéder à vos fichiers
from google.colab import drive
drive.mount('/content/drive')

# Installer les packages nécessaires
!pip install pydub
!apt-get install -y ffmpeg

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


#

**Note :** La première exécution de la cellule vous demandera d'autoriser l'accès à votre Google Drive. Cliquez sur le lien, connectez-vous et accepter les différentes étapes. (autorisez tout)

# Fonction de découpage

Lancez juste la cellule ci-dessous.

In [ ]:
from pydub import AudioSegment, silence
import os

def smart_split_audio(file_path, output_dir='/content/drive/MyDrive/IA pour maman', chunk_length_ms=60*60*1000, search_window_ms=60*1000, silence_thresh_db=-40):
    """
    Découpe un fichier audio autour de tranches d'une heure,
    en essayant de couper sur un silence proche de la limite.
    """
    audio = AudioSegment.from_file(file_path)
    os.makedirs(output_dir, exist_ok=True)

    chunks = []
    current_pos = 0
    total_duration = len(audio)
    part = 1

    while current_pos < total_duration:
        target_end = current_pos + chunk_length_ms
        if target_end > total_duration:
            target_end = total_duration

        # Chercher un silence dans la fenêtre autour de la cible
        search_start = max(current_pos + chunk_length_ms - search_window_ms, current_pos)
        search_end = min(current_pos + chunk_length_ms + search_window_ms, total_duration)

        segment_to_search = audio[search_start:search_end]
        silence_points = silence.detect_silence(segment_to_search,
                                                min_silence_len=700,
                                                silence_thresh=silence_thresh_db)

        if silence_points:
            # Prendre la première coupure silencieuse détectée
            silence_relative = silence_points[0][0]
            cut_point = search_start + silence_relative
        else:
            # Pas de silence détecté, on coupe brutalement
            cut_point = target_end

        chunk = audio[current_pos:cut_point]
        chunk_path = os.path.join(output_dir, f"audiopart_{part}.m4a")
        chunk.export(chunk_path, format="mp4")
        print(f"Chunk {part} sauvegardé : {chunk_path}")

        current_pos = cut_point
        part += 1

    print("✂️ Découpage terminé.")

### **Chargement du fichier**

**Indiquez ici le chemin vers votre fichier audio sur Google Drive.**

Pour récupérer le chemin :
1. Allez dans le panneau de gauche ("Fichiers") de Colab.
2. Cliquez sur l'icône Google Drive.
3. Naviguez jusqu'à votre fichier audio ("drive" -> "MyDrive" -> "IA pour maman"), faites un clic droit et sélectionnez "Copier le chemin d'accès".
4. Bien mettre le chemin d'accès entre guillemets (simple ou double peut importe)


**Exemple :**


audio_file = '/content/drive/MyDrive/IA pour maman/Meeting.m4a'

In [ ]:
audio_file='/content/drive/MyDrive/IA pour maman/B03.04.m4a'

### **Lancement du découpage**

**Lancer la cellule ci-dessous pour que la fonction de découpage fasse le traitement de l'audio et enregistre les fichiers dans le dossier "IA pour maman"**



**Attention :** Vérifier bien que le bon fichier audio est sélectionné ci-dessus.


**En cas d'erreur ou autre**, vous pouvez arrêter le traitement en appuyant sur le carré qui a remplacé le 'play' en haut à gauche de la cellule.**

.

**Pour convertir un autre fichier, remplacer le fichier ci-dessus par le chemin d'accès de votre nouveau fichier puis relancer la cellule ci-dessous**

In [ ]:
smart_split_audio(audio_file)

Chunk 1 sauvegardé : /content/drive/MyDrive/IA pour maman/audiopart_1.m4a
Chunk 2 sauvegardé : /content/drive/MyDrive/IA pour maman/audiopart_2.m4a
Chunk 3 sauvegardé : /content/drive/MyDrive/IA pour maman/audiopart_3.m4a
✂️ Découpage terminé.
